## QR Decomposition
Given a matrix
$$
\boldsymbol{A} = \begin{bmatrix}
2 & 0 & -1 \\
2 & 4 & -3 \\
3 & 2 & 4
\end{bmatrix},
$$

find its QR Decomposition using:
- Gram Schmidt method
- Householder method 

But first, let us calculate the answer beforehand using the built-in function $\texttt{np.linalg.qr(A)}$

In [48]:
import numpy as np
np.set_printoptions(suppress=True, precision=3)

A = np.array([
    [2, 0, -1],
    [2, 4, -3],
    [3, 2, 4]
], dtype=np.float64)

Q, R = np.linalg.qr(A)
print("Matrix Q is\n{}".format(Q))
print("Matrix R is\n{}".format(R))

Matrix Q is
[[-0.485  0.566 -0.667]
 [-0.485 -0.808 -0.333]
 [-0.728  0.162  0.667]]
Matrix R is
[[-4.123 -3.395 -0.97 ]
 [ 0.    -2.91   2.506]
 [ 0.     0.     4.333]]


### Part 1. Gram Schmidt method

Suppose that matrix $\boldsymbol{A}$ has the following form:
$$
\boldsymbol{A} = \left[ \boldsymbol{A}^{[1]},\boldsymbol{A}^{[2]},\dots,\boldsymbol{A}^{[n]} \right]
$$

We first define $\boldsymbol{Q}^{[1]} := \hat{\boldsymbol{A}}^{[1]}$. For each $k=2,\dots,n$ we evaluate:
$$
\boldsymbol{q}^{[k]} = \boldsymbol{A}^{[k]} - \sum_{i=1}^{k-1} \langle \boldsymbol{A}^{[k]},\boldsymbol{Q}^{[i]} \rangle \boldsymbol{Q}^{[i]}, \; \; \; \boldsymbol{Q}^{[k]} = \hat{\boldsymbol{q}}^{[k]}
$$

We define another matrix $\boldsymbol{R}$ as follows:
$$
\boldsymbol{R}_{i,k} = \begin{cases}
\langle \boldsymbol{A}^{[k]}, \boldsymbol{Q}^{[i]} \rangle, \; i < k \\
\|\boldsymbol{q}^{[k]}\|_2, \; i=k \\
0, i>k
\end{cases}
$$

This way, our matrix is decomposed as follows:
$$
\boldsymbol{A} = \boldsymbol{QR}
$$

In [53]:
def gram_schmidt_qr(A):
    """
    Input:
    A - square matrix
    
    Output:
    Q, R - matrices in QR decomposition
    """
    n = len(A) # Matrix size
    Q_cols = [A[:, 0] / np.linalg.norm(A[:, 0])] # Columns of matrix Q
    Q_lens = [np.linalg.norm(A[:, 0])] # Lengths ||Qi||
    
    # Finding Q_cols and Q_lens
    for k in range(1, n):
        Ak = A[:, k] # Taking the kth column of A
        qk = Ak - np.sum([Ak.dot(q)*q for q in Q_cols], axis=0) # Finding q_k according to formula above
        Q_cols.append(qk / np.linalg.norm(qk)) # Adding normalized q_k to the array of Q_cols
        Q_lens.append(np.linalg.norm(qk)) # Adding norm of q_k to the array of Q_lens
    
    Q = np.column_stack(Q_cols) # Building a matrix based on vector columns
    R = np.zeros(shape=(n, n)) # Initializing a matrix R
    
    # Finding matrix R
    for i in range(n):
        for k in range(n):
            if i > k: # R_{i,k} = 0 if i > k
                R[i][k] = 0
                continue
            # if i<k, R_{i,k} = <A^k, Q^i>, otherwise ||q_k||
            R[i][k] = A[:, k].dot(Q_cols[i]) if i < k else Q_lens[k]
    return Q, R

In [54]:
Q, R = gram_schmidt_qr(A)
print("Using Gram Schmidt method we obtain Q =\n{}".format(Q))
print("Using Gram Schmidt method we obtain R =\n{}".format(R))

Using Gram Schmidt method we obtain Q =
[[ 0.485 -0.566 -0.667]
 [ 0.485  0.808 -0.333]
 [ 0.728 -0.162  0.667]]
Using Gram Schmidt method we obtain R =
[[ 4.123  3.395  0.97 ]
 [ 0.     2.91  -2.506]
 [ 0.     0.     4.333]]


### Part 2. Reflection method

In [55]:
def reflection_qr(M):
    """
    Input:
    M - square matrix
    
    Output:
    Q, R - matrices in QR decomposition
    """
    n = len(M) # Size of matrix M
    A = M.copy() # Copying the matrix M
    Q = np.identity(n) # Initializing Q = E
    
    for k in range(n-1):
        v = A[:,k].copy() # Taking a kth column of matrix A
        v[:k] = 0 # Making 0 on all position below kth (or above if we consider v as a vector)
        v[k] = v[k] - np.linalg.norm(v, 2)  # From the element a_{k,k} we subtract norm of v
        v = v / np.linalg.norm(v, 2) # Normalizing vector v
        Q = Q - 2 * Q.dot(v.reshape(n,1)).dot(v.reshape(1,n)) # Finding Q_{k+1} = Q_k - 2Q_k*v*v^T
        A = A - 2 * v.reshape(n,1).dot(v.reshape(1,n)).dot(A) # Finding A_{k+1} = A_k - 2v*v^T*A_k
    return Q, A

In [56]:
Q, R = reflection_qr(A)
print("Using Reflection method we obtain Q =\n{}".format(Q))
print("Using Reflection method we obtain R =\n{}".format(R))

Using Reflection method we obtain Q =
[[ 0.485 -0.566 -0.667]
 [ 0.485  0.808 -0.333]
 [ 0.728 -0.162  0.667]]
Using Reflection method we obtain R =
[[ 4.123  3.395  0.97 ]
 [ 0.     2.91  -2.506]
 [ 0.     0.     4.333]]
